In [12]:
## set keras backend and image_dim_ordering
import os
os.environ['KERAS_BACKEND'] = 'tensorflow' # or 'tensorflow'
from keras import backend as K
K.set_image_dim_ordering('tf')

print K.image_dim_ordering(), K.backend()

tf tensorflow


In [13]:
from __future__ import division
import sys
sys.path.insert(0, '../MyPackages/')
sys.path.insert(0, '../MyPackages/Inception-v4/')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2, os, random, time
from tqdm import tqdm

from keras.applications.inception_v3 import preprocess_input, InceptionV3
from keras.applications.inception_v4 import InceptionV4
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.utils.visualize_util import plot
from inception_v4 import create_inception_v4

In [22]:
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_dim_ordering() == 'tf'

model = InceptionV4(input_tensor=input_tensor, weights=None, include_top=False)

In [18]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_448 (Convolution2D (None, 111, 111, 32)  864         input_6[0][0]                    
____________________________________________________________________________________________________
batchnormalization_448 (BatchNor (None, 111, 111, 32)  128         convolution2d_448[0][0]          
____________________________________________________________________________________________________
activation_448 (Activation)      (None, 111, 111, 32)  0           batchnormalization_448[0][0]     
___________________________________________________________________________________________

In [19]:
model_inceptionv4 = create_inception_v4(nb_classes=1001, load_weights=True)

Model weights loaded.


In [20]:
model_inceptionv4.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_597 (Convolution2D (None, 149, 149, 32)  864         input_7[0][0]                    
____________________________________________________________________________________________________
batchnormalization_597 (BatchNor (None, 149, 149, 32)  128         convolution2d_597[0][0]          
____________________________________________________________________________________________________
activation_597 (Activation)      (None, 149, 149, 32)  0           batchnormalization_597[0][0]     
___________________________________________________________________________________________

In [11]:
for l in model_inceptionv4.layers:
    print l
    w = l.get_weights()
    print [x.shape for x in w]

[]
[(32, 3, 3, 3)]
[(32,), (32,), (32,), (32,)]
[]
[(32, 32, 3, 3)]
[(32,), (32,), (32,), (32,)]
[]
[(64, 32, 3, 3)]
[(64,), (64,), (64,), (64,)]
[]
[(96, 64, 3, 3)]
[(96,), (96,), (96,), (96,)]
[]
[]
[]
[(64, 160, 1, 1)]
[(64,), (64,), (64,), (64,)]
[]
[(64, 64, 1, 7)]
[(64,), (64,), (64,), (64,)]
[]
[(64, 160, 1, 1)]
[(64, 64, 7, 1)]
[(64,), (64,), (64,), (64,)]
[(64,), (64,), (64,), (64,)]
[]
[]
[(96, 64, 3, 3)]
[(96, 64, 3, 3)]
[(96,), (96,), (96,), (96,)]
[(96,), (96,), (96,), (96,)]
[]
[]
[]
[(192, 192, 3, 3)]
[(192,), (192,), (192,), (192,)]
[]
[]
[]
[(64, 384, 1, 1)]
[(64,), (64,), (64,), (64,)]
[]
[(64, 384, 1, 1)]
[(96, 64, 3, 3)]
[(64,), (64,), (64,), (64,)]
[(96,), (96,), (96,), (96,)]
[]
[]
[]
[(96, 384, 1, 1)]
[(96, 64, 3, 3)]
[(96, 96, 3, 3)]
[(96, 384, 1, 1)]
[(96,), (96,), (96,), (96,)]
[(96,), (96,), (96,), (96,)]
[(96,), (96,), (96,), (96,)]
[(96,), (96,), (96,), (96,)]
[]
[]
[]
[]
[]
[(64, 384, 1, 1)]
[(64,), (64,), (64,), (64,)]
[]
[(64, 384, 1, 1)]
[(96, 64, 3, 3)

KeyboardInterrupt: 

In [ ]:
xin = model_inceptionv4.input
xoutm = model_inceptionv4.layers[-2].output
# xout = Dense(output_dim=1, init='he_normal')(xoutm)
model_aesthetics1 = Model(input=xin, output=xoutm)

In [ ]:
model_aesthetics1.summary()

In [ ]:
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_dim_ordering() == 'tf'

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [ ]:
w = w[2]
print w.shape

In [23]:
xinnew = Input(shape=(224,224,3))
x = xinnew
for i in model_inceptionv4.layers[1:-1]:
    print i
    x = i(x)
xoutnew = x

# xoutnew = model_aesthetics1(xinnew)
xoutnew = Dense(output_dim=1, init='he_normal')(xoutnew)
model_aesthetics1 = Model(input=xinnew, output=xoutnew)

ValueError: Input 0 is incompatible with layer batchnormalization_597: expected shape=(None, 149, 149, 32), found shape=(None, 111, 111, 32)

In [ ]:
print xin.get_shape()
print xoutm.get_shape()

In [ ]:
plot(model_inceptionv4, to_file='model.png')